In [1]:
# !pip3 install PyTelegramBotAPI
# !pip list

In [2]:
'''Импортируем необходимое для повышения разрешения'''
import torch
import esrinf 

In [ ]:
'''Импортируем необходимые функции для переноса стиля и генерации текстуры'''
from transfer import run_style_transfer, image_loader, save_image, generate_noise
import telebot
import numpy as np

'''Телеграм бот'''
bot_token = '5405399298:AAGW5Cm-MrJGaAl74N-QkvmvvJkqlyvZld8'
bot = telebot.TeleBot(bot_token, parse_mode='html')
'''Создаем модель повышения разрешения'''
sr_model, device = esrinf.make_srmodel()

'''Помощь по выбору режимов'''
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):    
    torch.cuda.empty_cache()    
    bot.send_message(message.from_user.id, "Выберите режим: /transfer, /texture или /super")

'''Функции для режимов '''
@bot.message_handler(commands=['super'])
def super_resolution(message):        
    bot.send_message(message.from_user.id, 'Выбран режим повышения разрешения, не давайте мне большие (>512pix) картинки')
    bot.send_message(message.from_user.id, 'Дайте мне изображение для повышения разрешения...')
    bot.register_next_step_handler(message, super_res)
           
    
@bot.message_handler(commands=['transfer'])
def style_transfer(message):    
    bot.send_message(message.from_user.id, 'Выбран режим переноса стиля')   
    bot.send_message(message.from_user.id, 'Дайте мне контент изображение...')   
    bot.register_next_step_handler(message, get_content_pic)
    
@bot.message_handler(commands=['texture'])
def style_transfer(message):    
    bot.send_message(message.from_user.id, 'Выбран режим генерации текстуры')   
    bot.send_message(message.from_user.id, 'Дайте мне изображение текстуры...')       
    bot.register_next_step_handler(message, get_texture)

'''Функция повышения разрешения, принимает картинку низкого разрешения, 
сохраняет ее как lowrez.jpg, вызывает процедуру повышения разрешения, которая 
создает и отправляет hirez.png'''
def super_res(message):
    try:      
        bot.send_message(message.from_user.id, "Изображение  получено.")
        bot.send_message(message.from_user.id, "Повышаем разрешение...")
        content_file_id = message.photo[-1].file_id
        content_path = "./images/lowrez.jpg"
        content_file_info = bot.get_file(content_file_id)
        content_downloaded_file = bot.download_file(content_file_info.file_path)
        with open(content_path,'wb') as new_file:
            new_file.write(content_downloaded_file)
    except:
        send_welcome(message)
        
    try:        
        esrinf.make_sr(sr_model, device)
        result = open('./images/hirez.png', 'rb')  
        bot.send_photo(message.chat.id, photo=result, caption='Готово!') 
    except Exception as e:        
        bot.send_message(message.from_user.id, e)  
#         send_welcome(message)       

    torch.cuda.empty_cache()
    send_welcome(message)
'''Функция получения контент изображения для переноса стиля, принимает картинку 
контента, сохраняет ее как content.jpg, регестрирует следующим шагом функцию 
получения стиля'''
def get_content_pic(message):    
    try:        
        bot.send_message(message.from_user.id, "Контент получен.")
        content_file_id = message.photo[-1].file_id
        content_path = "./images/content.jpg"
        content_file_info = bot.get_file(content_file_id)
        content_downloaded_file = bot.download_file(content_file_info.file_path)
        with open(content_path,'wb') as new_file:
            new_file.write(content_downloaded_file)                
        bot.send_message(message.from_user.id, "Дайте мне изображение стиля.")
        bot.register_next_step_handler(message, get_style_pic)        
    except:
        send_welcome(message)
    
'''Функция получения изображения стиля для переноса стиля, принимает картинку 
стиля, сохраняет ее как style.jpg, запускает функцию calculate_result, которая
считает результат
'''
def get_style_pic(message): 
    try:
        bot.send_message(message.from_user.id, "Стиль получен.")
        style_file_id = message.photo[-1].file_id
        style_path = "./images/style.jpg"
        style_file_info = bot.get_file(style_file_id)          
        style_downloaded_file = bot.download_file(style_file_info.file_path)
        with open(style_path,'wb') as new_file:
            new_file.write(style_downloaded_file)             
        send = bot.send_message(message.from_user.id, "Считаем результат(10сек)...")
        calculate_result(message, texture_gen = False)
    except Exception as e:
        print(e)
    send_welcome(message)
    
'''Функция получения изображения текстуры для генерации текстуры, принимает картинку 
текстуры, сохраняет ее как style.jpg, создает шум в качестве контента, сохраняет его как
content.jpg запускает функцию calculate_result с параметром texture_gen = True, которая 
считает результат, с несколько отличными от режима переноса стиля параметрами'''
def get_texture(message):
    try:        
        send = bot.send_message(message.from_user.id, "Текстура получена.")   
        style_file_id = message.photo[-1].file_id
        style_path = "./images/style.jpg"
        style_file_info = bot.get_file(style_file_id)          
        style_downloaded_file = bot.download_file(style_file_info.file_path)
        with open(style_path,'wb') as new_file:
            new_file.write(style_downloaded_file)                          
        content_img = generate_noise()
        content_path = "./images/content.jpg"
        content_img.save(content_path)        
        send = bot.send_message(message.from_user.id, "Считаем результат(около минуты)...")
        calculate_result(message, texture_gen = True)
    except:
        pass
    send_welcome(message)
    
'''Функция переноса стиля и генерации текстуры, считывает файлы style.jpg, 
content.jpg и вызывает run_style_transfer, которая вычисляет результат, отправляет его'''
def calculate_result(message, texture_gen = False):
    style_img = image_loader("./images/style.jpg")
    content_img = image_loader("./images/content.jpg")
    input_img = content_img.clone()

    result = run_style_transfer(content_img, style_img, 
                                input_img, num_steps=400, 
                                texture_gen = texture_gen)
    save_image(result, './images/result.png')
    result = open('./images/result.png', 'rb')      
    bot.send_photo(message.chat.id, photo=result, caption='Готово!')        

bot.enable_save_next_step_handlers(delay=1)    
bot.load_next_step_handlers()

bot.polling(non_stop=True)

Модель esrgan создана
